In [17]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score 
from sklearn.model_selection import  StratifiedShuffleSplit

In [18]:
data = pd.read_csv('./database.csv')
data.shape

(77, 13)

In [19]:
"""data.centroidX= (data.centroidX - data.centroidX.min()) / (data.centroidX.max() - data.centroidX.min())
data.centroidY= (data.centroidY - data.centroidY.min()) / (data.centroidY.max() - data.centroidY.min())
data.orientation = (data.orientation  - data.orientation.min()) / (data.orientation.max() - data.orientation.min())
data.perimeter= (data.perimeter - data.perimeter.min()) / (data.perimeter.max() - data.perimeter.min())
data.meanIntensity= (data.meanIntensity - data.meanIntensity.min()) / (data.meanIntensity.max() - data.meanIntensity.min())
data.extent= (data.extent - data.extent.min()) / (data.extent.max() - data.extent.min())
data.aspectRatio= (data.aspectRatio - data.aspectRatio.min()) / (data.aspectRatio.max() - data.aspectRatio.min())
data.area = (data.area - data.area.min()) / (data.area.max() - data.area.min())
data.equivalentDiameter = (data.equivalentDiameter - data.equivalentDiameter.min()) / (data.equivalentDiameter.max() - data.equivalentDiameter.min())"""
data = data.drop(columns=['previousCancerDiagnosis', 'dicomPatientId'])
data.head()

,label,area,perimeter,aspectRatio,centroidX,centroidY,equivalentDiameter,extent,meanIntensity,orientation,eccentricity
0,0,334,226.681241,1.636364,134.697605,423.724551,20.621882,0.421717,0.320923,-1.429113,0.811104
1,0,334,226.681241,1.636364,134.697605,423.724551,20.621882,0.421717,0.320923,-1.429113,0.811104
2,0,342,150.953319,0.500000,228.985380,441.038012,20.867389,0.527778,0.496996,-0.096871,0.874032
3,0,138,61.698485,0.800000,36.362319,428.847826,13.255454,0.766667,0.489031,0.155235,0.602152
4,1,108,53.319805,1.000000,122.601852,213.833333,11.726460,0.750000,0.321387,0.929097,0.418157


In [20]:
featuresColumns = [x for x in data.columns if x not in 'label']
stf = StratifiedShuffleSplit(n_splits=1, test_size=10, random_state=42)
trainIdx, testIdx = next(stf.split(data[featuresColumns], data['label']))

xTrain = data.loc[trainIdx, featuresColumns]
yTrain = data.loc[trainIdx, 'label']

xTest = data.loc[testIdx, featuresColumns]
yTest = data.loc[testIdx, 'label']

In [21]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(xTrain, yTrain)

KNeighborsClassifier(n_neighbors=1)

In [22]:
def measureErrors(yTrue, yGuess, label):
    return pd.Series({
        'accuracy': accuracy_score(y_true=yTrue, y_pred=yGuess),
        'precision': precision_score(y_true=yTrue, y_pred=yGuess),
        'recall': recall_score(y_true=yTrue, y_pred=yGuess),
        'roc': roc_auc_score(yTrue,yGuess)
        },name=label
    )

In [23]:
trainPrediction = knn.predict(xTrain)
testPrediction = knn.predict(xTest)

trainAndTestErrors = pd.concat([measureErrors(yTrain, trainPrediction, 'train'), measureErrors(yTest, testPrediction, 'test')], axis=1)

trainAndTestErrors

,train,test
accuracy,1.0,0.900
precision,1.0,1.000
recall,1.0,0.750
roc,1.0,0.875


In [24]:
import pickle;
pickle.dump(knn, open('dt.sav', 'wb'));